In [1]:
import pandas as pd
import numpy as np
import psycopg2
np.random.seed(42)
exams = pd.read_csv('exams.csv')

In [2]:
print(exams.head(1))
print(exams.columns)

   gender race/ethnicity parental level of education     lunch  \
0  female        group D                some college  standard   

  test preparation course  math score  reading score  writing score  
0               completed          59             70             78  
Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')


In [3]:
exams.drop(['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course'], axis=1, inplace=True)

In [4]:
exams

,math score,reading score,writing score
0,59,70,78
1,96,93,87
2,57,76,77
3,70,70,63
4,83,85,86
...,...,...,...
995,77,77,71
996,80,66,66
997,67,86,86
998,80,72,62


In [31]:
operands = ['>', '<', '=', '>=', '<=', '!=']
operands_one_hots = np.diag(np.ones(6))
nr_cols = 6     # total nr of [int] cols in both tables
cols_names = ['math score', 'reading score', 'writing score', 'math score2', 'reading score2', 'writing score2']

''' Makes n queries randomly + uniformly

tables: 2 dataframes
table_names: list of two possible vals - 2 tables
n: number of queries to be generated
max_nr_preds: max number of predicates per query
'''
def generate_queries(tables, table_names, n = 20000, max_nr_preds = 1):
    def generate_query():

        sql_query = ''
        sql_query_vec = []

        # if join the first bit of sql query is 1 --> that is indicator of join
        # [join_bit]
        sql_query_vec.append(np.random.randint(0, 2))
        table_ind = np.random.randint(0, 2)

        # [join_bit, tb1_bit, tb2_bit]
        if sql_query_vec[0]:
            # You are joining --> select * from tb1, tb2 where tb1 == tb2
            sql_query = f'SELECT * FROM {table_names[0]}, {table_names[1]} WHERE {table_names[0]} = {table_names[1]} AND'
            sql_query_vec.extend([1, 1]) # <- if join, dont need AND_bit or OR_bit
        else:
            sql_query = f'SELECT * FROM {table_names[table_ind]} WHERE'
            if table_ind:
                sql_query_vec.extend([0, 1])
            else:
                sql_query_vec.extend([1, 0])

        # nr_predicates = {1,2}
        nr_predicates = 1 if sql_query_vec[0] else np.random.randint(1, max_nr_preds + 1)

        # Need to edit this to be the columns of the tables in equation

        column = np.random.choice(range(0, tables[table_ind].shape[1]), nr_predicates) if table_ind == 0 else np.random.choice(range(tables[table_ind].shape[1], nr_cols), nr_predicates)
        # column = np.random.choice(['math score', 'reading score', 'writing score'], nr_predicates)

        # sign is either array of len 1 or 2
        # 6 operands
        sign = np.random.choice(range(6), nr_predicates)

        # Each loop should end in something like 
        # [..., <columns_bits~nr_cols> <sign_bits~6> <value real>]
        for predicate in range(nr_predicates):
            value = np.random.randint(np.min(tables[table_ind][cols_names[column[predicate]]]), np.max(tables[table_ind][cols_names[column[predicate]]]))
            if predicate == 0:
                sql_query += f' {cols_names[column[predicate]]} {operands[sign[predicate]]} {value}'
            else:
                if np.random.randint(0, 2):
                    sql_query += f' AND {cols_names[column[predicate]]} {operands[sign[predicate]]} {value}'
                    sql_query_vec.extend([1, 0])
                elif sql_query_vec[0]:  # if u r joining, we treat it as no extra AND
                    sql_query += f' AND {cols_names[column[predicate]]} {operands[sign[predicate]]} {value}'
                    sql_query_vec.extend([0, 0])
                else:
                    sql_query += f' OR {cols_names[column[predicate]]} {operands[sign[predicate]]} {value}'
                    sql_query_vec.extend([0, 1])
                    

            tmp = [0] * 6
            tmp[column[predicate]] = 1
            sql_query_vec.extend(tmp)

            tmp = [0] * 6
            tmp[sign[predicate]] = 1
            sql_query_vec.extend(tmp)

            sql_query_vec.append(value)

        if nr_predicates == 1:
            sql_query_vec.extend([0] * (nr_cols + 6 + 1 + 2)) 

        return sql_query, sql_query_vec

    sqlQuery = []
    vectorQuery = []
    for i in range(n):
        sql_query, sql_query_vec = generate_query()
        sqlQuery.append(sql_query)
        vectorQuery.append(sql_query_vec)

    
    return sqlQuery, np.array(vectorQuery)

In [35]:
# for a single tabled query, have format: 
# [<join_bit=0> <tb1_bit> <tb2_bit> <column id [6 bits one hot]> <sign id [6 bits ont hit]> <value num> <AND_bit> <OR_bit> <column id [6 bits one hot]> <sign id [6 bits ont hit]> ] or just 0s if predicate = 1
# for joins, we have:
# select * from tb1, tb2 where tb1.col = tb2.col AND tb1.col/tb2.col <sign> val
# [<join_bit=1> <tb1_bit=1> <tb2_bit=1> <column id [6 bits one hot]> <sign id [6 bits ont hit]> <value num> <AND_bit=0> <OR_bit=0> <0s> ]
# note: only AND statement is allowed, but in the vectorization, we treat it as if the AND is not there (one predicate problem)
sqlQuery, vectorQuery = generate_queries([exams, 100 * exams.rename({'math score' : 'math score2', 'reading score':'reading score2', 'writing score':'writing score2'}, axis=1)], ['exams', 'exams2'], n = 10, max_nr_preds = 2)

In [36]:
sqlQuery

['SELECT * FROM exams2 WHERE math score2 != 3827',
 'SELECT * FROM exams2 WHERE reading score2 = 5772 OR reading score2 > 3647',
 'SELECT * FROM exams, exams2 WHERE exams = exams2 AND math score2 = 2159',
 'SELECT * FROM exams, exams2 WHERE exams = exams2 AND writing score < 81',
 'SELECT * FROM exams WHERE writing score > 15 AND math score < 25',
 'SELECT * FROM exams WHERE writing score != 21 AND writing score < 30',
 'SELECT * FROM exams, exams2 WHERE exams = exams2 AND math score < 15',
 'SELECT * FROM exams, exams2 WHERE exams = exams2 AND math score2 <= 8658',
 'SELECT * FROM exams WHERE writing score = 34',
 'SELECT * FROM exams, exams2 WHERE exams = exams2 AND math score2 >= 3148']

In [37]:
vectorQuery

array([[   0,    0,    1,    0,    0,    0,    1,    0,    0,    0,    0,
           0,    0,    0,    1, 3827,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    1,    0,    0,    0,    0,    1,    0,    0,    0,
           1,    0,    0,    0, 5772,    0,    1,    0,    0,    0,    0,
           1,    0,    1,    0,    0,    0,    0,    0, 3647],
       [   1,    1,    1,    0,    0,    0,    1,    0,    0,    0,    0,
           1,    0,    0,    0, 2159,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   1,    1,    1,    0,    0,    1,    0,    0,    0,    0,    1,
           0,    0,    0,    0,   81,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    1,    0,    0,    0,    1,    0,    0,    0,    1,    0,
           0,    0,    0,    0,   15,    1,    0,    1,    0,    0,    0,
        

In [13]:
# user is postgres
# port is 5433

In [16]:
hostname = 'localhost'
database = 'postgres'
username = 'postgres'
pwd = None # need to add this
port_id = 5433
connection, cur = None, None

try:
  # need to download postgres and supply your own info 
  # with password and stuff
  connection = psycopg2.connect(host = hostname,
                                dbname = database,
                                user = username,
                                password = pwd,
                                port = port_id)

  cur = connection.cursor()

  cur.execute('explain (format json, analyze) select * from exams_sample')
  result = cur.fetchone()[0][0]
  print(result['Plan']['Plan Rows'])
  print(result['Plan']['Actual Rows'])

except Exception as error:
  print(error)
finally:
  if cur:
    cur.close()
  if connection:
    connection.close()


1000
1000
